In [ ]:
!pip install accelerate transformers einops datasets peft bitsandbytes torch

In [ ]:
import torch
from datasets import load_dataset, Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForLanguageModeling, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model
import os

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2", trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/phi-2",
    device_map={"":0},
    trust_remote_code=True,
    quantization_config=bnb_config
)

In [ ]:
print(model)

In [ ]:
config = LoraConfig(
    r=16,
    lora_alpha=16,
    #target_modules=["q_proj","k_proj","v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
model.print_trainable_parameters()

In [ ]:
def tokenize(sample):
    model_inps =  tokenizer(sample["text"], padding=True, truncation=True, max_length=512)
    return model_inps

In [ ]:
!pip install scikit-learn

In [ ]:
import pandas as pd

from sklearn.model_selection import train_test_split
dataset_name='data.csv'
df = pd.read_csv(dataset_name)
#train, test = train_test_split(df, test_size=0.2)

In [ ]:
data_df = df
data_df["text"] = data_df[["user", "assistant"]].apply(lambda x: "question: " + str(x["user"]) + " answer: " + str(x["assistant"]), axis=1)
data = Dataset.from_pandas(data_df)
tokenized_data = data.map(tokenize, batched=True, desc="Tokenizing data", remove_columns=data.column_names)
tokenized_data

In [ ]:
training_arguments = TrainingArguments(
        output_dir=".",
        per_device_train_batch_size=4,
        gradient_accumulation_steps=1,
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        save_strategy="epoch",
        logging_steps=50,
        max_steps=10000,
        num_train_epochs=3,
        push_to_hub=True
    )

In [ ]:
!pip install huggingface_hub

In [ ]:
pip install --upgrade jupyterlab jupyterlab-git

In [ ]:
from huggingface_hub import interpreter_login
interpreter_login()

In [ ]:
 !pip install torch==2.1.0 pytorch-lightning==1.9.4 accelerate==0.21.0 tokenizers==0.13.3 transformers

In [ ]:
model.enable_input_require_grads()

In [ ]:
trainer = Trainer(
    model=model,
    train_dataset=tokenized_data,
    args=training_arguments,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
trainer.train()

In [ ]:
from peft import PeftModel
from transformers import AutoModelForCausalLM
import torch
model = AutoModelForCausalLM.from_pretrained("microsoft/phi-2", trust_remote_code=True, torch_dtype=torch.float32)
peft_model = PeftModel.from_pretrained(model, "aissatoubalde/lab", from_transformers=True)
model = peft_model.merge_and_unload()
model

In [ ]:
model.push_to_hub('aissatoubalde/lab')